In [2]:
df_exp = pd.read_csv('/home/yamanishi/project/trip_recommend/data/jalan/spot/experience_light.csv')
df_review = pd.read_csv('/home/yamanishi/project/trip_recommend/data/jalan/review/review_all_period_.csv')

In [3]:
from datetime import datetime
from tqdm import tqdm
tqdm.pandas()
def correct_time(time):
    time = time[:time.index('月')+1]
    return time
def to_datetime(time):
    return datetime.strptime(time, '%Y年%m月')
df_review['visit_time_correct'] = df_review['visit_time'].progress_apply(correct_time)
df_review['visit_time_correct'] = df_review['visit_time_correct'].progress_apply(to_datetime)


100%|██████████| 3565959/3565959 [00:57<00:00, 62186.46it/s] 


In [ ]:
from collections import defaultdict
duplicate_spots = set()
review_series = {}
for s in df_exp['spot_name']:
    if s in review_series:
        duplicate_spots.add(s)
    review_series[s] = [0]*2
#print(duplicate_spots)
for s in duplicate_spots:
    del review_series[s]
print(len(review_series))

for s,t in tqdm(zip(df_review['spot'], df_review['visit_time_correct'])):
    if s not in duplicate_spots and s in review_series:
        review_series[s][t.year-1950]+=1

In [4]:
df_review_before_covid = df_review[('2019-1-1'<df_review['visit_time_correct'])&(df_review['visit_time_correct']<'2020-4-1')]

In [5]:
df_review_after_covid = df_review[('2020-4-1'<df_review['visit_time_correct'])&(df_review['visit_time_correct']<'2021-7-1')]

In [55]:
print(len(df_review_before_covid))
print(len(df_review_after_covid))

229024
85041


In [ ]:
df_reivew_before_covid['series']

In [7]:
from collections import defaultdict
duplicate_spots = set()
review_series = {}
for s in df_exp['spot_name']:
    if s in review_series:
        duplicate_spots.add(s)
    review_series[s] = [0]*73
#print(duplicate_spots)
for s in duplicate_spots:
    del review_series[s]
print(len(review_series))

for s,t in tqdm(zip(df_review['spot'], df_review['visit_time_correct'])):
    if s not in duplicate_spots and s in review_series:
        review_series[s][t.year-1950]+=1

41963


3565959it [00:10, 339796.98it/s]


In [8]:
df_tmp = pd.DataFrame.from_dict(review_series, orient='index')
df_tmp.head()

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
函館山,0,0,0,0,0,0,0,0,0,0,...,448,800,729,1048,569,288,138,24,30,34
旭山動物園,0,0,0,0,0,0,0,0,0,0,...,384,761,624,814,458,272,102,48,26,38
六花亭本店,0,0,0,0,0,0,0,0,0,0,...,102,263,318,402,225,150,97,48,30,26
神威岬,0,0,0,0,0,0,0,0,0,0,...,76,151,162,236,115,67,36,13,12,11
ニトリ観光果樹園,0,0,0,0,0,0,0,0,0,0,...,2,1,6,5,5,3,1,7,4,9


In [9]:
df_tmp.columns =df_tmp.columns+1950

In [10]:
df_tmp.loc[:, 2005:].head()

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
函館山,1,0,4,9,191,240,224,307,448,800,729,1048,569,288,138,24,30,34
旭山動物園,0,0,9,23,165,245,214,280,384,761,624,814,458,272,102,48,26,38
六花亭本店,0,0,1,6,41,66,65,71,102,263,318,402,225,150,97,48,30,26
神威岬,0,0,1,2,29,25,29,37,76,151,162,236,115,67,36,13,12,11
ニトリ観光果樹園,0,0,0,0,0,0,2,1,2,1,6,5,5,3,1,7,4,9


In [11]:
cand = set()
for s, r0,r1,r2 in zip(df_tmp.index, df_tmp[2010], df_tmp[2011], df_tmp[2012]):
    if r0+r1+r2>0:
        cand.add(s)
len(cand)

20003

In [12]:
df_tmp['spot_name'] = df_tmp.index

In [22]:
th_year = 2016
df_cand = df_tmp[df_tmp['spot_name'].isin(cand)]
print(len(df_cand))
df_tmp_before = df_cand.loc[:, :th_year]
df_tmp_after = df_cand.loc[:, th_year:]
df_before_sum=df_tmp_before.sum(axis=1)
df_after_sum = df_tmp_after.sum(axis=1)
th=50
print(df_before_sum)
print(len(df_before_sum[df_before_sum>th]))
print(len(df_before_sum[df_after_sum>th]))
before_popular_spots = set(df_before_sum[df_before_sum>=th].index)
before_unpopular_spots = set(df_before_sum[df_before_sum<th].index)
after_popular_spots = set(df_after_sum[df_after_sum>=th].index)
after_unpopular_spots = set(df_after_sum[df_after_sum<th].index)
pp=(before_popular_spots&after_popular_spots)
pu=(before_popular_spots&after_unpopular_spots)
uu=(before_unpopular_spots&after_unpopular_spots)
up=(before_unpopular_spots&after_popular_spots)
print(len(pp), len(pu), len(uu), len(up))

20003
函館山                  4003
旭山動物園                3519
六花亭本店                1335
神威岬                   749
ニトリ観光果樹園               17
                     ... 
鍋島陣屋跡                   8
前浜海水浴場（長崎県南島原市）        10
【2021年度中止】半元キャンプ場       4
ドンドン渕                   4
対馬藩お船江跡                 5
Length: 20003, dtype: int64
6062
5570
4830 1321 13049 803


/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/study_group/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  import sys


In [14]:
spot_idx = {spot:i for i,spot in enumerate(df_exp['spot_name'])}

In [15]:
mask = [False]*len(df_exp)
label = [-1]*len(df_exp)
for s in uu:
    id = spot_idx[s]
    mask[id] = True
    label[id] = 0
for s in up:
    id = spot_idx[s]
    mask[id] = True
    label[id] = 1
for s in pu:
    id = spot_idx[s]
    mask[id] = True
    label[id]= 2

for s in pp:
    id = spot_idx[s]
    mask[id] = True
    label[id] = 3

In [16]:
np.save('/home/yamanishi/project/trip_recommend/model/popularity_final/data/mask.npy', mask)
np.save('/home/yamanishi/project/trip_recommend/model/popularity_final/data/label.npy', label)

In [17]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
label = np.load('/home/yamanishi/project/trip_recommend/model/popularity_final/data/label.npy')
label_ = label[label!=-1]
print(len(label_))
skf = StratifiedKFold(n_splits=10, shuffle=True)
split=np.array([0]*len(label_))
for i,(train_index, test_index) in enumerate(skf.split(label_, label_)):
    split[test_index]=i

20003


In [18]:
np.save('/home/yamanishi/project/trip_recommend/model/popularity_final/data/split.npy', split)

In [19]:
label=np.load('/home/yamanishi/project/trip_recommend/model/popularity_final/data/label.npy')

In [32]:
mask=np.load('/home/yamanishi/project/trip_recommend/model/popularity_final/data/mask.npy')
split=np.load('/home/yamanishi/project/trip_recommend/model/popularity_final/data/split.npy')


1

In [45]:
train_mask = mask.nonzero()[0][split>1]
valid_mask = mask.nonzero()[0][split==0]
test_mask = mask.nonzero()[0][split==1]
np.save('/home/yamanishi/project/trip_recommend/model/popularity_final/data/train_mask.npy', train_mask)
np.save('/home/yamanishi/project/trip_recommend/model/popularity_final/data/valid_mask.npy', valid_mask)
np.save('/home/yamanishi/project/trip_recommend/model/popularity_final/data/test_mask.npy', test_mask)

In [48]:
np.load('/home/yamanishi/project/trip_recommend/model/popularity_final/data/test_mask.npy')

array([    1,     2,    27, ..., 42776, 42820, 42848])